# Ozone vertical profile

## Import libraries

In [ ]:
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import download

## Define request

In [ ]:
collection_id = "satellite-ozone-v1"
common_request = {
    "processing_level": "level_3",
    "variable": "mole_concentration_of_ozone_in_air",
    "vertical_aggregation": "vertical_profiles_from_limb_sensors",
}
sensor_keys = ["sensor", "version"]
limb_requests = [
    {
        "sensor": "gomos",
        "version": "v0001",
        "start": "2002-08",
        "stop": "2011-12",
    },
    {
        "sensor": "mipas",
        "version": "v0001",
        "start": "2002-07",
        "stop": "2012-04",
    },
    {
        "sensor": "sage_2",
        "version": "v0001",
        "start": "1984-10",
        "stop": "2005-08",
    },
    {
        "sensor": "sciamachy",
        "version": "v0001",
        "start": "2002-08",
        "stop": "2012-03",
    },
    {
        "sensor": "smr",
        "version": "v0001",
        "start": "2001-07",
        "stop": "2014-08",
    },
    {
        "sensor": "cmzm",
        "version": "v0008",
        "start": "1984-10",
        "stop": "2022-12",
        "rename": {
            "merged_ozone_concentration": "ozone_concentration",
            "uncertainty_of_merged_ozone_concentration": "mean_uncertainty_estimate",
        },
    },
]
assert all(
    set(sensor_keys) | {"start", "stop"} <= set(request) for request in limb_requests
)

## Transform function

In [ ]:
def add_time_dim(ds):
    time_str = ds.attrs["time_coverage_start"][:6]
    time = pd.to_datetime(time_str, format="%Y%m")
    return ds.expand_dims(time=[time])

## Download

In [ ]:
datasets = []
for limb_request in limb_requests:
    request = common_request | {key: limb_request[key] for key in sensor_keys}
    requests = download.update_request_date(
        request, limb_request["start"], limb_request["stop"], stringify_dates=True
    )
    sensor = "-".join([request[sensor_key] for sensor_key in sensor_keys])
    print(f"{sensor = }")
    ds = download.download_and_transform(
        collection_id,
        requests,
        chunks={"year": 1},
        preprocess=add_time_dim,
    )
    ds = ds.rename(limb_request.get("rename", {})).expand_dims(sensor=[sensor])
    datasets.append(ds)
ds = xr.concat(datasets, "sensor")

## Quick and dirty plot

In [ ]:
time = "2005-08"
ds["ozone_concentration"].sel(time=time).plot(col="sensor", col_wrap=3)